In [1]:
with open('digitalcorp.txt') as f:
    lines = f.readlines()

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import hashlib

# PART 1

In [5]:
lines = [i.replace("\n", "") for i in lines]

In [6]:
df = pd.read_table('digitalcorp.txt', sep=',')

In [7]:
passwords = list(pd.read_table('rockyou.txt', header=None)[0])

In [8]:
#passwords

In [9]:
dc = {}
for st in passwords:
    inp = bytes(st, 'utf-8')
    dc[(hashlib.sha512(inp).hexdigest())] = st

In [10]:
cracked = []
for hashed_pw in df.hash_of_password:
    if hashed_pw in dc:
        cracked.append(dc[hashed_pw])

In [11]:
df['cracked'] = cracked

In [12]:
df

,username,hash_of_password,cracked
0,Creed,f286725e49415dfeb4546d96bba3dc88e9d9d096ff4e5b...,cocacola
1,Meredith,7f8e33ea99bca90fe54caf134c9258e160945600e06849...,50cent
2,Stanley,d8e44d39c4e438dc668c4e105c536a2a90630f51b4d2bf...,patrick
3,Phyllis,8512de11f6042ae4128256c8e6c1bfb68ee50434ab09ae...,newyork


# PART 2

In [13]:
sdf = pd.read_table('salty-digitalcorp.txt', sep=',')

In [14]:
sdf

,username,salt,hash_outcome
0,Kevin,6aa2bbb690bdff92,c6050538cc65f06079a3e17abe415cb31c2ef1d5c8a18d...
1,Angela,42ccd5c0f6455812,9cfe4177d36350de4dbacb41ec735b5e07492415e9aa62...
2,Oscar,d61e65884e0d3203,6be96652538e7ae4835e071e101117279c1a8e0f37a562...
3,Darryl,1d5dbcb692042997,cfedd51710a577ee51277cdcac2045a8483275f4b61a27...


In [15]:
dcs = {}
for i in range(len(sdf)):
    salt = (sdf.iloc[i].salt)
    #pw = sdf.iloc[i].hash_outcome
    
    for st in passwords:
        inp = bytes(st + salt, 'utf-8')
        inp2 = bytes(salt + st, 'utf-8')
        dcs[(hashlib.sha512(inp).hexdigest())] = st
        dcs[(hashlib.sha512(inp2).hexdigest())] = st

In [16]:
cracked_salted = []
for hashed_pw in sdf.hash_outcome:
    if hashed_pw in dcs:
        cracked_salted.append(dcs[hashed_pw])

In [17]:
cracked_salted

['tinkerbell', 'chrisbrown', 'chivas', 'eminem']

In [18]:
sdf['cracked'] = cracked_salted

In [19]:
sdf

,username,salt,hash_outcome,cracked
0,Kevin,6aa2bbb690bdff92,c6050538cc65f06079a3e17abe415cb31c2ef1d5c8a18d...,tinkerbell
1,Angela,42ccd5c0f6455812,9cfe4177d36350de4dbacb41ec735b5e07492415e9aa62...,chrisbrown
2,Oscar,d61e65884e0d3203,6be96652538e7ae4835e071e101117279c1a8e0f37a562...,chivas
3,Darryl,1d5dbcb692042997,cfedd51710a577ee51277cdcac2045a8483275f4b61a27...,eminem


# PART 3

In [20]:
kdf = pd.read_table('keystreching-digitalcorp.txt', sep=',')

In [21]:
kdf

,username,salt,hash_outcome
0,Jim,6aa2bbb690bdff01,3b3a82652d9f3a5a3ed894665f106ffe36b845490d588d...
1,Pam,42ccd5c0f6455810,817367d6e9b8e4219fa5c78d85ca75e9ed1dc6f64eb747...
2,Dwight,d61e65884e0d3299,483075aa9c8b9298eb882d22c4a54bd522f8694d44b91f...
3,Michael,1d5dbcb692042924,3e4d009b62c74dd4a14085deb5463bec3b7f8cf2ed3a38...


In [22]:
salt = kdf.salt.iloc[0] #Jim

In [23]:
def crack_with_salt(salt):
    xi = ''
    exp_ls = []
    dict_2000 = {}

    for pw in passwords:
        xi = ''
        xi1 = ''
        xi2 = ''
        xi3 = ''

        exp_ls = []
        for i in range(2000): #TODO: change
            xi = bytes(xi + salt + pw, 'utf-8') #xi + pw + salt #pw + salt + xi #xi + salt + pw #xi + salt + pw
            #xi + pw + salt, xi + salt + pw, pw + salt + xi, 
            xi = hashlib.sha512(xi).hexdigest()
            exp_ls.append(xi)

            xi1 = bytes(pw + xi1 + salt, 'utf-8')
            xi1 = hashlib.sha512(xi1).hexdigest()
            exp_ls.append(xi1)

            #xi2 = bytes(xi2 + salt + pw, 'utf-8')
            #xi2 = hashlib.sha512(xi2).hexdigest()
            #exp_ls.append(xi2)

            xi3 = bytes(salt + xi3 + pw, 'utf-8')
            xi3 = hashlib.sha512(xi3).hexdigest()
            exp_ls.append(xi3)


        dict_2000[pw] = exp_ls
    
    crack_list_2000 = []
    key = ''

    for hashed in kdf.hash_outcome:
        for key in dict_2000:
            cur_ls = dict_2000[key]

            if hashed in cur_ls:
                crack_list_2000.append(key)
                return key
    return key

In [24]:
cracked_list_p3 = []

for salt in kdf.salt.iloc[:]:
    cr = crack_with_salt(salt)
    cracked_list_p3.append(cr)

In [25]:
cracked_list_p3

['hottie', 'cutiepie', 'angelica', 'superstar']

In [26]:
kdf['cracked'] = cracked_list_p3

In [27]:
'''crack_list_2000 = []

for hashed in kdf.hash_outcome:
    for key in dict_2000:
        cur_ls = dict_2000[key]
        
        if hashed in cur_ls:
            crack_list_2000.append(key)
    '''

'crack_list_2000 = []\n\nfor hashed in kdf.hash_outcome:\n    for key in dict_2000:\n        cur_ls = dict_2000[key]\n        \n        if hashed in cur_ls:\n            crack_list_2000.append(key)\n    '

In [28]:
kdf

,username,salt,hash_outcome,cracked
0,Jim,6aa2bbb690bdff01,3b3a82652d9f3a5a3ed894665f106ffe36b845490d588d...,hottie
1,Pam,42ccd5c0f6455810,817367d6e9b8e4219fa5c78d85ca75e9ed1dc6f64eb747...,cutiepie
2,Dwight,d61e65884e0d3299,483075aa9c8b9298eb882d22c4a54bd522f8694d44b91f...,angelica
3,Michael,1d5dbcb692042924,3e4d009b62c74dd4a14085deb5463bec3b7f8cf2ed3a38...,superstar
